In [1]:
import os
import pandas as pd
import numpy as np
import scipy
import datetime
from sklearn.model_selection import train_test_split
os.chdir('C:/Users/Paperspace/Desktop/Kaggle-Competitions/FraudDetection')

In [3]:
train = pd.read_csv('train.csv')
#train, test = train_test_split(train, test_size = 0.30, random_state = 42)
test = pd.read_csv('test.csv')

In [4]:
def dataPreProcessTime(df):
    df['click_time'] = (pd.to_datetime(df['click_time']) - datetime.datetime(2017,1,1,0,0,0)).dt.seconds
    return df

train = dataPreProcessTime(train)
test = dataPreProcessTime(test)

In [7]:
from scipy.sparse import csr_matrix
del test
X_train = train[['ip',
 'app',
 'device',
 'os',
 'channel',
 'click_time']]

y_train = train[['is_attributed']]

del train

X_test = test[['ip',
 'app',
 'device',
 'os',
 'channel',
 'click_time']]

del test

#y_test = test[['is_attributed']]

MemoryError: 

In [7]:
print(X_train.shape)
print(X_test.shape)

(100000, 6)
(18790469, 6)


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import recall_score
import time

In [ ]:
def recall_lgb(preds, d_train):
    labels = d_train.get_label()
    preds = np.where((preds > 0.5), 1, 0)
    recs = recall_score(preds, labels)
    return 'recall', recs, True

In [ ]:
params = {'boosting' : 'gbdt', 'learning_rate' : 0.02, 'application' : 'binary', 'max_depth' : 7, 'num_leaves' : 26, 
         'data_random_seed' : 42, 'bagging_fraction' : 0.4, 'nthread' : 8, 'metric' : 'auc', 'is_unbalanced' : True}

In [ ]:
start = time()
train_X, valid_X, train_y, valid_y = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)
del X_train
del y_train

d_train = lgb.Dataset(train_X, label = train_y)
d_valid = lgb.Dataset(valid_X, label = valid_y)
d_test = lgb.Dataset(X_test)

watchlist = [d_train, d_valid]

model = lgb.train(param = )

In [8]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)

C:\Users\dhruv\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [ ]:
pred = clf.predict(X_test)

In [ ]:
import pickle

with open('outfile', 'wb') as fp:
    pickle.dump(pred, fp)